## 重新分群

- 要重新做記得要重新跑一遍 `Organising-materials`

In [15]:
import pandas as pd 
from shapely.geometry import Point
import geopandas as gpd
import numpy as np
from collections import Counter
import math
import matplotlib.pyplot as plt
import seaborn as sns
## import my modules
import sys
sys.path.append("..\\modules")
import convert
import aggregation

## for k means

from sklearn.cluster import KMeans

In [16]:
Taiwan_village_variable = gpd.read_file("..\\..\\Roaming-Dogs-Data\\@Taiwan_village_variable\\Taiwan_village_variable.shp",encoding="big5")
Taiwan_village_variable = Taiwan_village_variable.set_index("VILLCODE")


In [17]:
Taiwan_village_variable.shape

(7939, 54)

- 使用依照人口密度(P_DEN)、專科以上人口比例(high_rat)、老化指數(AGING_IDX)等變數，以群分析（Kmeans）作為全台灣都市化程度分為四層。
- 原本只是照區做分群，現在照村里，單位更細
- 原本有農業人口比例，但是我找不到！
- 原本用老年人口比例，但這次改用老化指數



In [18]:
col_kmeans = ["P_DEN", "high_rat", "AGING_IDX"]

尋找nan
- [[link]](https://blog.csdn.net/u012387178/article/details/52571725)

In [19]:
df_kmeans = Taiwan_village_variable[col_kmeans]
print(df_kmeans.isnull().any()) # 都有nan

not_null_list = []

for i in col_kmeans:
    not_null_0 = df_kmeans[df_kmeans[i].isnull().values != True].index
    not_null_list.append(not_null_0)
    
not_null = not_null_list[0]

for n  in not_null_list[1:]:
    not_null = not_null.intersection(n)


print(len(not_null))

P_DEN        True
high_rat     True
AGING_IDX    True
dtype: bool
7658


In [20]:
df_kmeans = df_kmeans.loc[not_null]
print(df_kmeans.shape)
df_kmeans.isnull().any()

(7658, 3)


P_DEN        False
high_rat     False
AGING_IDX    False
dtype: bool

In [21]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler().fit(df_kmeans)
df_X_scaled = scaler.transform(df_kmeans)
df_X_scaled = np.insert(df_X_scaled.T,0,df_kmeans.index,axis= 0)
df_X_scaled = pd.DataFrame(dict(zip(["VILLCODE"]+col_kmeans, df_X_scaled)))
df_X_scaled = df_X_scaled.set_index("VILLCODE")
df_X_scaled.index = [int(x) for x in df_X_scaled.index]
df_X_scaled.index = [str(x) for x in df_X_scaled.index]

In [22]:
most_com = Counter(df_kmeans.index).most_common(40)

most_com_villcode = []

for i in range(40):
    if float(most_com[i][1]) >1:
        most_com_villcode.append(most_com[i][0])


In [23]:
print(most_com_villcode) # 確定沒有重複的村里

[]


### Kmeans

#### Elbow Method plot
- 中文教學 [[link]](https://medium.com/@pahome.chen/clustering%E6%B1%BA%E5%AE%9A%E5%88%86%E7%BE%A4%E6%95%B8%E7%9A%84%E6%96%B9%E6%B3%95-abedc1d81ccb)

In [24]:
# distortions = []
# K = range(2,10)
# for k in K:
#     kmeanModel = KMeans(init='random',n_clusters=k,random_state=100,  max_iter=10000, n_init=100)
#     kmeanModel.fit(df_X_scaled)
#     distortions.append(kmeanModel.inertia_)

In [25]:
# plt.figure(figsize=(16,8))
# plt.plot(K, distortions, 'bx-')
# plt.xlabel('k')
# plt.ylabel('Distortion')
# plt.title('The Elbow Method showing the optimal k')
# plt.show()

In [26]:
col_Cluster = []
for i in range(2,6):
    kmeans = KMeans(init='random',random_state=10000, n_clusters=i, max_iter=10000, n_init=100).fit(df_X_scaled)
    df_kmeans[f"Clu_new_{i}"] = kmeans.labels_
    df_X_scaled[f"Clu_new_{i}"] = kmeans.labels_
    col_Cluster.append(f"Clu_new_{i}")   

In [27]:
print(type(Taiwan_village_variable.index[0]),type(df_X_scaled.index[0]))

<class 'str'> <class 'str'>


In [28]:
print(df_X_scaled[col_Cluster])


             Clu_new_2  Clu_new_3  Clu_new_4  Clu_new_5
64000130006          0          1          1          3
64000130008          0          1          1          3
64000100010          0          1          1          3
64000100013          0          1          1          3
10013020003          0          1          1          3
...                ...        ...        ...        ...
10008110015          0          0          2          4
67000290019          0          1          1          3
10008010030          0          1          1          3
10008010010          0          1          1          3
10008010029          1          2          0          1

[7658 rows x 4 columns]


In [29]:
for i in col_Cluster:
    print(df_X_scaled[col_kmeans+[i]].groupby(by = i).mean())
    print(Counter(df_X_scaled[i]))

              P_DEN  high_rat  AGING_IDX
Clu_new_2                               
0         -0.512157 -0.545183   0.206501
1          0.953499  1.014986  -0.384450
Counter({0: 4982, 1: 2676})
              P_DEN  high_rat  AGING_IDX
Clu_new_3                               
0         -0.556197 -1.013381   1.743016
1         -0.500803 -0.432523  -0.163086
2          0.952922  1.015683  -0.385250
Counter({1: 4016, 2: 2675, 0: 967})
              P_DEN  high_rat  AGING_IDX
Clu_new_4                               
0          0.363871  1.087791  -0.414777
1         -0.500803 -0.432523  -0.163086
2         -0.556197 -1.013381   1.743016
3          2.501879  0.826067  -0.307606
Counter({1: 4016, 0: 1938, 2: 967, 3: 737})
              P_DEN  high_rat  AGING_IDX
Clu_new_5                               
0          2.501879  0.826067  -0.307606
1          0.363871  1.087791  -0.414777
2         -0.563980 -1.063235   9.130697
3         -0.500803 -0.432523  -0.163086
4         -0.556016 -1.012220  

In [30]:
Taiwan_village_variable_new = pd.concat([Taiwan_village_variable, df_X_scaled[col_Cluster]], axis=1)

In [31]:
Taiwan_village_variable_new.insert(0, "VILLCODE", Taiwan_village_variable_new.index, allow_duplicates=True)

In [32]:
Taiwan_village_variable_new.to_file(driver='ESRI Shapefile', \
    filename='..\\..\\Roaming-Dogs-Data\@Taiwan_village_variable\\Taiwan_village_variable.shp', encoding="Big5")


In [33]:
path ='..\\..\\Roaming-Dogs-Data\\zz_期末報告\\'
N = pd.read_csv(path+'109_推估結果_村里.csv' ,encoding="big5")
Taiwan_village_variable = gpd.read_file("..\\..\\Roaming-Dogs-Data\\@Taiwan_village_variable\\Taiwan_village_variable.shp",encoding="big5", index = True)

C:\Users\ChenWenRu\anaconda3\lib\site-packages\geopandas\array.py:85: ShapelyDeprecationWarning: __len__ for multi-part geometries is deprecated and will be removed in Shapely 2.0. Check the length of the `geoms` property instead to get the  number of parts of a multi-part geometry.
  aout[:] = out


In [34]:
Taiwan_village_variable.columns

Index(['VILLCODE', 'COUNTYNAME', 'TOWNNAME', 'VILLNAME', 'VILLENG', 'COUNTYID',
       'COUNTYCODE', 'TOWNID', 'TOWNCODE', 'NOTE', 'Cluster', 'high_rat',
       'mid_rat', 'low_rat', 'high_edu', 'mid_edu', 'low_edu', 'Hospital_1',
       'Hospital_2', 'Hospital_3', 'Hospital_4', 'M_F_RAT', 'P_H_CNT', 'P_DEN',
       'DEPENDENCY', 'YOUN_DEP', 'OLD_DEP', 'AGING_IDX', 'H_CNT', 'P_CNT',
       'M_CNT', 'F_CNT', 'Ele', 'Ele_stu', 'Junior', 'Junior_stu', 'Senior',
       'Senior_stu', 'TaxPayer', 'Income_tot', 'Income_mea', 'Income_med',
       'Income_Q1', 'Income_Q3', 'Income_sta', 'Income_CV', 'Temple',
       'Train-stat', 'Train-crow', 'Hospital', 'Clinic', 'Area_sqkm', 'X', 'Y',
       'Clu_new_2', 'Clu_new_3', 'Clu_new_4', 'Clu_new_5', 'geometry'],
      dtype='object')

In [35]:
Variable_XY = Taiwan_village_variable.merge(N[['COUNTYNAME','TOWNNAME','VILLNAME',"Nr"]], left_on = ['COUNTYNAME','TOWNNAME','VILLNAME'], right_on = ['COUNTYNAME','TOWNNAME','VILLNAME'], how = 'inner')
print(Variable_XY.shape)
Variable_XY = gpd.GeoDataFrame(Variable_XY, geometry = 'geometry', crs =Taiwan_village_variable.crs )

Variable_XY.to_file(driver='ESRI Shapefile', \
    filename='..\\..\\Roaming-Dogs-Data\\@Taiwan_sampling_village_variable\\Taiwan_sampling_village_variable.shp', encoding="Big5")

(616, 60)
